### 1. Getting Data

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None) # exibir todas as colunas do meu df
pd.set_option("display.max_seq_items", None) # exibir todas as linhas no meu for

In [6]:
#data = 'https://raw.githubusercontent.com/bruno-lima98/ml-zoomcamp-homework/refs/heads/main/01-classes/03-classification/telco_data.csv'
#!wget $data -O telco_data.csv

In [3]:
df = pd.read_csv('telco_data.csv')

### 2. Data Prep

In [4]:
# Padronizamos a escrita das colunas, títulos e informações

df.columns = df.columns.str.lower().str.replace(' ','_')
categorical_colums = list(df.dtypes[df.dtypes == 'object'].index)

for i in categorical_colums:
    df[i] = df[i].str.lower().str.replace(' ','_')

In [5]:
# Ajustamos a coluna totalcharges que é número mas estava como string por possuir espaços vazios

df['totalcharges'] = pd.to_numeric(df['totalcharges'].str.replace('_','0'))

In [6]:
# Ajustamos nossa variável target binária para 0/1 no lugar yes/no

df['churn'] = (df['churn'] == 'yes').astype('int')

In [7]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,no,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


### 3. Setting up Validation Framework

In [8]:
from sklearn.model_selection import train_test_split
# Separamos o nosso dataset em Treino, Validação e Teste usando uma biblioteca diretamente

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [10]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [11]:
len(df_full_train), len(df_train), len(df_test), len(df_val)

(5634, 4225, 1409, 1409)

In [12]:
# Resetamos os indíces dos DFs apenas para não ficar embaralhado

df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [13]:
y_full_train = df_full_train['churn'].values
y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test['churn'].values

In [14]:
# del df_full_train['churn']
del df_train['churn']
del df_val['churn']
del df_test['churn']

### 4. EDA

In [ ]:
df_full_train.isnull().sum()
# Verificamos que não temos mais valores nulos no df

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [24]:
global_churn_rate = df_full_train['churn'].value_counts(normalize=True)[1]
# Essa é uma forma de filtrar os valores. Calculei o % de cada categoria e depois chamei o valor da categoria que vale 1.

round(global_churn_rate,2)

np.float64(0.27)

In [27]:

numerical = ['tenure','monthlycharges','totalcharges']
categorical = [
                'gender', 'seniorcitizen', 'partner', 'dependents',
                'phoneservice', 'multiplelines', 'internetservice',
                'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
                'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
                'paymentmethod']

In [48]:
for i in categorical:
    print(f'{i:<17} | Total de categorias: {df_full_train[i].nunique()} | Categorias: {list(df_full_train[i].unique())}')

gender            | Total de categorias: 2 | Categorias: ['male', 'female']
seniorcitizen     | Total de categorias: 2 | Categorias: [np.int64(0), np.int64(1)]
partner           | Total de categorias: 2 | Categorias: ['yes', 'no']
dependents        | Total de categorias: 2 | Categorias: ['yes', 'no']
phoneservice      | Total de categorias: 2 | Categorias: ['yes', 'no']
multiplelines     | Total de categorias: 3 | Categorias: ['no', 'yes', 'no_phone_service']
internetservice   | Total de categorias: 3 | Categorias: ['no', 'dsl', 'fiber_optic']
onlinesecurity    | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
onlinebackup      | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
deviceprotection  | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
techsupport       | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
streamingtv       | Total de categorias: 3 | Categorias: ['no_internet_ser

### 5. Featuring Engineering

In [55]:
male_churn_rate = df_full_train[df_full_train['gender']=='male']['churn'].mean()
female_churn_rate = df_full_train[df_full_train['gender']=='female']['churn'].mean()

print(f'Taxa de Churn Global = {global_churn_rate:.4f} | Taxa (homens) = {male_churn_rate:.4f} | Taxa (mulheres) = {female_churn_rate:.4f}')

# Podemos ver que as taxas entre homens e mulheres são bem parecidas, indicando que talvez a variável de sexo não importe tanto

Taxa de Churn Global = 0.2700 | Taxa (homens) = 0.2632 | Taxa (mulheres) = 0.2768


In [56]:
partner_churn = df_full_train[df_full_train['partner']=='yes']['churn'].mean()
no_partner_churn = df_full_train[df_full_train['partner']=='no']['churn'].mean()

print(f'Taxa de Churn Global = {global_churn_rate:.4f} | Taxa (Com Parceiro) = {partner_churn:.4f} | Taxa (Sem Parceiro) = {no_partner_churn:.4f}')

# Podemos ver que há uma diferença considerável entre o churn em pessoas com parceiro e sem parceiro, indicando que essa talvez possa ser uma variável importante

Taxa de Churn Global = 0.2700 | Taxa (Com Parceiro) = 0.2050 | Taxa (Sem Parceiro) = 0.3298


In [76]:
a = ((df_full_train.groupby('paymentmethod')['churn'].mean() / global_churn_rate - 1).round(4)*100)
a[1]

/tmp/ipykernel_49344/3938576978.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a[1]


np.float64(-39.129999999999995)

In [82]:
for i in categorical:
    a = (df_full_train.groupby(i)['churn'].mean() / global_churn_rate - 1).round(4)*100
    print(f'{i}')
    for j in range(len(a)):
        print(f'- {a.index[j]}: {a.values[j].round(2)}')

gender
- female: 2.54
- male: -2.5
seniorcitizen
- 0: -10.26
- 1: 53.12
partner
- no: 22.17
- yes: -24.05
dependents
- no: 16.22
- yes: -38.63
phoneservice
- no: -10.61
- yes: 1.14
multiplelines
- no: -4.65
- no_phone_service: -10.61
- yes: 7.69
internetservice
- dsl: -28.75
- fiber_optic: 57.49
- no: -71.18
onlinesecurity
- no: 55.92
- no_internet_service: -71.18
- yes: -43.24
onlinebackup
- no: 49.77
- no_internet_service: -71.18
- yes: -19.53
deviceprotection
- no: 46.64
- no_internet_service: -71.18
- yes: -14.65
techsupport
- no: 55.17
- no_internet_service: -71.18
- yes: -40.76
streamingtv
- no: 26.99
- no_internet_service: -71.18
- yes: 12.13
streamingmovies
- no: 25.54
- no_internet_service: -71.18
- yes: 13.82
contract
- month-to-month: 59.91
- one_year: -55.34
- two_year: -89.53
paperlessbilling
- no: -36.26
- yes: 25.26
paymentmethod
- bank_transfer_(automatic): -37.71
- credit_card_(automatic): -39.13
- electronic_check: 68.87
- mailed_check: -28.19
